# Feature Selection for Categorical Features using Iterative Hard Thresholding

Feature selection is a very important task in machine learning tasks. Finding a relevant subset of features improves generalization, robustness to noise, and convergence to targets. A prominent technique in feature selection *Iterative Hard Thresholding (IHT)*. Although IHT is a powerful technique for feature selection, it is not designed for categorical features. In this notebook, I will give a walkthrough on **how to extend IHT to categorical datasets**. For validation, I will compare IHT to other machine learning techniques that also perform feature selections such as LASSO and Random Forest.


# Iterative Hard Thresholding



Consider a dataset $(\mathbf{X_i}, y_i)$ for $i = [n] = 1,2,3 ... n$, where $\mathbf{X_i} \in \mathbb{R}^m$ and $y_i \in \mathbb{R}.$ We seek to find a k-sparse vector $\mathbf{B} \in  \mathbb{R}^m$ with the following objective: 


$$ \min_{\|\mathbf{B}\|_0 = k} \| \mathbf{y} - f(\mathbf{X} ;\mathbf{B}) \|
$$

where $\mathbf{X} \in \mathbb{R}^{n \times m}, \mathbf{y} \in  \mathbb{R}^{n} $ denote the data matrix and the response vector. The $l_0$ norm denotes the number of non-zero elements in $\mathbf{B}$ and $f$ is a given task-specific function. 





Intuitively, learning $\mathbf{B}$ is equivalent to learning a set of $k$ relevant features that best recover the response vector. Thus, by solving this optimization problem, one can identify relevant features. 

IHT assumes there exists a linear relationship between a subset of the measured variable and the response vector , and seeks to retain the top-k values of the entire feature vector after each iteration. Let $\mathbf{B}^0 = \mathbf{0}$, IHT follows the update rule:

$$
\mathbf{B}^{t+1} =  \mathbf{H}_k(\mathbf{B}^{t} - \lambda \bigtriangledown_{\mathbf{B}^t} L(f(\mathbf{X} ;\mathbf{B}),y))
$$

In [1]:
from jax import grad
import jax.numpy as NP
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Dataset 

The dataset used in this work to validate our model was downloaded from [UCL Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Molecular+Biology+(Splice-junction+Gene+Sequences)). The data described Splice-junction Gene Sequences. The Splice junctions are points on a DNA sequence at which 'superfluous' DNA is removed during the process of protein creation in higher organisms.  The problem posed in this dataset is to recognize, given a sequence of DNA, the boundaries between exons (the parts of the DNA sequence retained after splicing) and introns (the parts of the DNA sequence that are spliced out). This problem consists of two subtasks: recognizing exon/intron boundaries (referred to as EI sites), and recognizing intron/exon boundaries (IE sites). (In the biological community, IE borders are referred to a "acceptors" while EI borders are referred to as "donors".) 

The data includes 3190 sample with 60 sequential DNA nucleotide positions as features .The data are classfied into 3 classes: EI, IE, and Neither. Each of these fields is almost always filled by one of {A, C, G, T}. Other characters indicate ambiguity among the standard characters according to the following table:

			character	    meaning
			---------	----------------
			    D		  A or G or T
			    N           A or G or C or T
			    S                C or G
			    R		     A or G

In [2]:
datafile ="molecularData.csv"
labelfile= "molecularLabel.csv"
df=pd.read_csv(datafile,header=None)
print("The shape of data is: ",df.shape)
print("Show example of dataset")
df.head(10)

The shape of data is:  (3190, 60)
Show example of dataset


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,C,C,A,G,C,T,G,C,A,T,...,A,G,C,C,A,G,T,C,T,G
1,A,G,A,C,C,C,G,C,C,G,...,G,T,G,C,C,C,C,C,G,C
2,G,A,G,G,T,G,A,A,G,G,...,C,A,C,G,G,G,G,A,T,G
3,G,G,G,C,T,G,C,G,T,T,...,G,G,T,T,T,T,C,C,C,C
4,G,C,T,C,A,G,C,C,C,C,...,C,C,T,T,G,A,C,C,C,T
5,C,A,G,A,C,T,G,G,G,T,...,G,A,G,A,C,C,A,C,A,G
6,C,C,T,T,T,G,A,G,G,A,...,G,T,G,G,C,C,G,C,C,A
7,C,C,C,T,C,G,T,G,C,G,...,G,G,T,C,G,T,G,G,G,G
8,T,G,G,C,G,A,C,T,A,C,...,G,C,T,C,C,A,G,T,C,C
9,A,A,G,C,T,G,A,C,A,G,...,G,A,G,G,G,T,G,A,G,A


 read data into the numpy array format

In [3]:
data = np.genfromtxt(datafile, dtype= str, delimiter = ",")
label= np.genfromtxt(labelfile, dtype = str, delimiter = ",")

In [4]:
label

array(['EI', 'EI', 'EI', ..., 'N', 'N', 'N'], dtype='<U2')

In [5]:
uniqueClasses = np.unique(label).tolist()
numClasses = len(uniqueClasses)
print("The classes are:",uniqueClasses)
print("The number of classes:",numClasses)

The classes are: ['EI', 'IE', 'N']
The number of classes: 3


# One Hot Encoding Categorical Features
One-hot encoding is an extremely common data transformation performed after indexing categorical variables.
OneHotEncoder, which will convert each distinct value to a Boolean flag (1 or 0) as a component in a vector。
One hot encoding creates new (binary) columns, indicating the presence of each possible value from the original data.
In this dataset, features are not given as continuous values but categorical. For example each column could have different possiable features['A', 'C','G', 'T']. We use column 0 as a example,

In [6]:
df.iloc[:,0:1].head(10)

,0
0,C
1,A
2,G
3,G
4,G
5,C
6,C
7,C
8,T
9,A


In [7]:
print("All the possible values in column 0 are:",df[0].unique())

All the possible values in column 0 are: ['C' 'A' 'G' 'T' 'D']


So One hot encoding will creates new (binary) columns, indicating the presence of each possible value from the original data, show as follwing:
![image info](./Onehot.png)

Now，we run following code to encoding our data, the feature name will show as following and filled by binary data 0 or 1

In [35]:
OHC = OneHotEncoder().fit(data)
OHCL = OneHotEncoder().fit(label.reshape(-1,1))

/Users/Junya/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


# Experiment Setting 

classW: The array holds the weight class for each class

classWL: The array holds the weights for each sample

In [9]:
classW = class_weight.compute_class_weight('balanced', np.unique(label), label)
classWL = np.array([classW[uniqueClasses.index(i)] for i in label]).reshape(-1,1) #Calculate weight class for each sample
label = LabelEncoder().fit_transform(label)
X_train, X_test, y_train, y_test = train_test_split(
    data, label, test_size=0.3, random_state=42)

Split data into training set and testing set

In [46]:
classCount = list(map(len, OHC.categories_)) #Number of all possible values for each categorical feature
indices = np.cumsum(classCount) # For indexing between coefficient and categorical features
DIM = indices[-1]
indices = np.insert(indices,0, 0)
X_trainC = OHC.transform(X_train).toarray()
X_testC = OHC.transform(X_test).toarray()
dataC = OHC.transform(data).toarray()
X_current, Y_current, weight =  None, None, None #Placeholder for batch data
num_train = X_train.shape[0]

Here, I set up the parameter for this model.

In [47]:
epoch = 2000
lr = 0.1
s = 50

In [48]:
def aggregateFeature(gradients, numSplit):
    gradients = gradients.flatten()
    sumList = []
    current = 0
    for i in numSplit:
        sumList.append(np.mean(np.abs(gradients[current: current + i ])))
        current = current + i
    return sumList
def thresholding(coeff):
    copyCof = coeff[:]
    coeff = np.sum(np.abs(np.array(coeff)), axis = 0) 
    sum_coeff = aggregateFeature(coeff, classCount)
    rankingBest = np.argsort(np.abs(sum_coeff)).ravel()[-s:]
    if rankingBest.shape[0] < s:
        print("rankBest less than s features")
    selected = set([i for j in rankingBest for i in range(indices[j], indices[j+1])]) #List of selected coefficient
    coeff = coeff.flatten()
    notSelected = list(set(range(len(coeff))).difference(selected))
    copyCof[:,notSelected] = 0
    return copyCof

In [49]:
def generateBatch():
    #int(num_train / 5)
    index = np.random.choice(num_train, size =int(num_train / 1), replace =  False)
    global X_current
    global Y_current
    global weight
    X_current = X_trainC[index]
    Y_current = y_train[index].reshape(-1,1)
    weight = classWL[index]

In [50]:
def regression(coeff):  # Define the softmax function
  y = NP.exp(NP.dot(X_current, coeff.T))
  s = NP.expand_dims(NP.sum(y, axis = 1), 1)
  y = y / s
  Y_currentT = OHCL.transform(Y_current).toarray()
  label_logprobs = NP.multiply(NP.log(y) , Y_currentT) #+ NP.multiply(NP.log(1 - y) , (1 - Y_currentT))
  label_logprobs = weight * label_logprobs
  return -NP.mean(label_logprobs)

  
def regressionTest(coeff):  # Define a function
  y = np.exp(np.dot(X_testC, coeff.T))
  s = np.expand_dims(np.sum(y, axis = 1), 1)
  y = y / s 
  return y

# IHT Model Result

In [51]:
cof = np.zeros((numClasses, DIM))
grad_regression  = grad(regression)
for i in range(epoch):
    generateBatch()
    trainError =  regression(cof)
    acc = accuracy_score(np.argmax(regressionTest(cof), axis = 1) , y_test)
    if i % 100 == 0:
        print("Training error:", trainError, " | Accuracy:",acc )
    gradient = np.array(grad_regression(cof))
    cof = thresholding(cof - lr * gradient)

Training error: 0.4223118  | Accuracy: 0.25914315569487983
Training error: 0.17073604  | Accuracy: 0.9456635318704284
Training error: 0.11760215  | Accuracy: 0.9487983281086729
Training error: 0.09478419  | Accuracy: 0.9508881922675027
Training error: 0.08186467  | Accuracy: 0.955067920585162
Training error: 0.073409416  | Accuracy: 0.9571577847439916
Training error: 0.067366384  | Accuracy: 0.9571577847439916
Training error: 0.06278567  | Accuracy: 0.9582027168234065
Training error: 0.059164897  | Accuracy: 0.9592476489028213
Training error: 0.056212064  | Accuracy: 0.9582027168234065
Training error: 0.05374511  | Accuracy: 0.9582027168234065
Training error: 0.05164423  | Accuracy: 0.9582027168234065
Training error: 0.049827036  | Accuracy: 0.9582027168234065
Training error: 0.048234776  | Accuracy: 0.9571577847439916
Training error: 0.046824478  | Accuracy: 0.9561128526645768
Training error: 0.045563687  | Accuracy: 0.9561128526645768
Training error: 0.04442762  | Accuracy: 0.9582027

# Baseline Performance

The performance of the classification models is assessed based on classification accuracy (ACC). 

In [52]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(dataC, label, random_state=1, test_size = 0.3)
        

clf = linear_model.SGDClassifier(loss ='log', penalty = 'l1')

clf.fit(X_train, y_train)


print("The accuracy score of LASSO: ",accuracy_score(y_test, clf.predict(X_test)))


clf = RandomForestClassifier()

clf.fit(X_train, y_train)


print("The accuracy score of Random Forest: ",accuracy_score(y_test, clf.predict(X_test)))

The accuracy score of LASSO:  0.9373040752351097
The accuracy score of Random Forest:  0.910135841170324


/Users/Junya/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/Users/Junya/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
